### Data transformation

This is the second automatically graded exercise for JODA. The objective here is to get our hands dirty with data.

The context of this particular analysis is a fictional company that routinely runs different machine learning operations.

We have generated a dataset that has the following columns or properties (to be engineered into features):

- Date
- Department
- ML Task ID
- ML Method
- Task Category
- Model Complexity (Parameters)
- Training Data Size (GB)
- Training Duration (Hours)
- Hardware Used
- Energy Consumption (kWh)
- CO2 Emissions (Kg)
- Cloud Provider

Moreover, there is a secondary dataset that includes information about the energy sources for the different cloud providers:

- Cloud Provider
- Green Energy

TODO: Import the needed packages

In [2]:
import pandas as pd

In [18]:
df_co2 = pd.read_excel('data/co2-emissions.xlsx')
df_providers = pd.read_excel('data/cloud-providers.xlsx')

df_final = df_co2.merge(df_providers, how='left', on='Cloud Provider')

Date                              object
Department                        object
ML Task ID                        object
ML Method                         object
Task Category                     object
Model Complexity (Parameters)    float64
Training Data Size (GB)          float64
Training Duration (Hours)        float64
Hardware Used                     object
Energy Consumption (kWh)         float64
CO2 Emissions (Kg)               float64
Cloud Provider                    object
Green Energy                      object
dtype: object


Aggregate the data to department level. That is, each row should represents the aggregated values for each department. Notice that you do not need to aggregate each different column, only the ones instructed explicitly.

In [22]:
df_department_level = df_final.groupby('Department').agg('CO2 Emissions (Kg)')

print(df_department_level)

Calculate the total of CO2 emissions for each department

In [23]:
df_department_level.sum()

Department
Customer Support    12565.569898
Finance             13568.637182
Human Resources     15256.236043
Marketing           12821.756125
Operations          15004.901708
R&D                 14644.874294
Name: CO2 Emissions (Kg), dtype: float64

Rename CO2 emission column to co2_emissions_kg

In [24]:
df_final.rename(columns={'CO2 Emissions (Kg)': 'co2_emissions_kg'}, inplace=True)

Create a function that picks the most common value among in a Pandas Series object

In [25]:
def pick_most_frequent(values):
    mode_series = values.mode()
    
    # If multiple modes are present, return the first mode
    most_common = mode_series.iloc[0] if not mode_series.empty else None
    
    return most_common

pick_most_frequent(pd.Series(['A', 'B', 'B', 'C']))

'B'

Pick the most frequent ML method for each department.

In [27]:
most_frequent_ml_method_per_department = df_final.groupby('Department')['ML Method'].agg(pick_most_frequent)
print(most_frequent_ml_method_per_department)

Department
Customer Support    Linear Regression
Finance                           RNN
Human Resources                   RNN
Marketing               Decision Tree
Operations                Transformer
R&D                               RNN
Name: ML Method, dtype: object


Make sure that the rows are sorted according to CO2 emissions in a way that the department with the largest emissions is first.

In [33]:
total_co2_emissions_per_department = df_final.groupby('Department')['co2_emissions_kg'].sum()

# Convert the result to a DataFrame
df_emissions_per_department = total_co2_emissions_per_department.reset_index()

# Sort the DataFrame by CO2 emissions in descending order
df_emissions_per_department_sorted = df_emissions_per_department.sort_values(by='co2_emissions_kg', ascending=False)

# Display the sorted DataFrame
print(df_emissions_per_department_sorted)

         Department  co2_emissions_kg
2   Human Resources      15256.236043
4        Operations      15004.901708
5               R&D      14644.874294
1           Finance      13568.637182
3         Marketing      12821.756125
0  Customer Support      12565.569898


Calculate the CO2 emissions for each department in different Green Energy categories. That is, the resulting dataframe will have as many colums as there are values for Green Energy.

In [35]:
pivot_table = df_final.pivot_table(index='Department', columns='Green Energy', values='co2_emissions_kg', aggfunc='sum')

print(pivot_table)

Green Energy            Green       Hybrid      Unknown
Department                                             
Customer Support  1463.697220  1425.139532  9676.733146
Finance           2991.539843  2427.085307  8150.012032
Human Resources   2423.439560  3431.874604  9400.921879
Marketing         2282.675323  2923.163567  7615.917236
Operations        2253.368372  3779.599277  8971.934059
R&D               3028.493500  2280.103568  9336.277226


Next, let's try to do something a bit more difficult. That is, calculate department CO2 emissions per energy type.

One way to achieve this is to use pivot_table() function to create a separate dataframe with the new columns and join (using merge()) that to the main dataframe. We are sure there are even more clever ways.

TODO: Include the specified columns to the result dataframe, one per each energy type.

In [38]:
pivot_table = pd.pivot_table(df_final, values='co2_emissions_kg', index='Department', columns='Green Energy', aggfunc='sum')

df_result = pd.merge(df_final, pivot_table, how='left', on='Department')

print(df_result)

           Date        Department ML Task ID          ML Method  \
0    2024-02-01         Marketing     Task_1                RNN   
1    2024-02-01        Operations     Task_2  Linear Regression   
2    2024-02-01               R&D     Task_3      Decision Tree   
3    2024-02-01  Customer Support     Task_4      Decision Tree   
4    2024-02-02           Finance     Task_5        Transformer   
..          ...               ...        ...                ...   
995  2024-10-06   Human Resources   Task_996        Transformer   
996  2024-10-07         Marketing   Task_997                RNN   
997  2024-10-07        Operations   Task_998      Decision Tree   
998  2024-10-07               R&D   Task_999  Linear Regression   
999  2024-10-07  Customer Support  Task_1000  Linear Regression   

            Task Category  Model Complexity (Parameters)  \
0    Image Classification                   2.008306e+06   
1              Regression                   1.053744e+03   
2     Machine T

In [39]:
df_providers['Green Energy'].value_counts()

Green Energy
Unknown    3
Green      1
Hybrid     1
Name: count, dtype: int64

Save the results

In [40]:
import os

def ensure_folder_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

ensure_folder_exists('results')

Folder 'results' created.


In [45]:
# df_result.to_excel('results/department_co2.xlsx', index=False)
df_final.to_pickle('results/department_co2.pkl')